# Model builder
## Life Cycle Assessment of ``{Operational Energy Moerschberg for 1 year: 1}``
###[CONTEXT]
The model relies on datapackages to ensure replicability of the calculation.


In [ ]:
"combinatorial"
"sequential"

In [146]:
##Uncomment to restore everything

# bd.projects.delete_project('ei38-teaching-25', delete_dir=True)

'default'

In [148]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing as bwp
import numpy as np
from pathlib import Path


if 'ei38-teaching-25' not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")
# bd.projects.delete_project("moeschpond1", delete_dir=True)
# if 'moeschpond1' not in bd.projects:    
#     backup="brightway2-project-moeschpond1-backup.27-October-2022-09-50AM.tar.gz"
#     premise_path  = "/srv/scratch/u.2/" + backup
#     bi.restore_project_directory(premise_path)
# bd.projects.set_current('moeschpond1')
bd.projects.set_current('ei38-teaching-25')


Restoring project backup archive - this could take a few minutes...


In [149]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei 3.8 cutoff

In [150]:
# if "foreground" in bd.databases:
#     del bd.databases["foreground"]
    
foreground_importer = bi.ExcelImporter("./data/inputs/foreground.xlsx")
for act in foreground_importer.data:
    # print(act["name"])
    for exc in act["exchanges"]:
        if "classifications" in exc:
            exc.pop("classifications")
        if "properties" in exc:
            exc.pop("properties")   
foreground_importer.apply_strategies()
foreground_importer.match_database("biosphere3", fields=("name", "unit", "categories"))
foreground_importer.match_database("ei 3.8 cutoff", fields=("name", "unit", "location"))
foreground_importer.statistics()
foreground_importer.write_database()

Extracted 1 worksheets in 0.02 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 7.26 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
11 datasets
151 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 

In [151]:
fu = {bd.get_activity(database="foreground", name = "Operationnal energy demand"):1}

In [5]:
# ## Creating datapackage for each method
# method_hint = "IPCC"
# data_objs_methods = []
# methods = [method for method in bd.methods if method_hint in method[0]]
# for method in methods:
#     _, objs, __ = bd.prepare_lca_inputs(method=method)
#     data_objs_methods += objs

In [152]:
### it returns functional_unit, data_objects, and a mapping dictionary
fu, objects, remapping_dicts = bd.prepare_lca_inputs(fu, method=('IPCC 2013', 'climate change', 'GWP 100a'))


In [158]:
for obj in objects:
    print(obj.metadata["name"])

biosphere3
ei_3.8_cutoff
foreground
ipcc-2013cg.bd5af3f67229a1cc291b8ecb7f316fcf.zip


In [164]:
data, _ = objects[2].get_resource("foreground_technosphere_matrix.data")
indices, _ = objects[2].get_resource("foreground_technosphere_matrix.indices")
flip, _ = objects[2].get_resource("foreground_technosphere_matrix.flip")



'Silicon, in ground' (kilogram, None, ('natural resource', 'in ground'))

In [7]:
database_name = "ei 3.8 cutoff"
wood_pellet = bd.get_activity(
    database=database_name, 
    name='wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future', 
    unit = "megajoule"
    )

In [8]:
wood_pellet.id

19364

In [9]:
bd.get_activity(
    id=19364
    )

'wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future' (megajoule, CH, None)

In [10]:
import json
with open("data/inputs/dict_id.json") as data:
    dicto = json.load(data)

In [11]:
dicto

{'wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future': 19364,
 'operation, solar collector system, Cu flat plate collector, one-family house, for hot water': 8302,
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted': 18352,
 'electricity production, wind, 1-3MW turbine, onshore': 10319,
 'electricity production, hydro, run-of-river': 6599,
 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014': 20079}

In [12]:
my_id =1_000_000
my_node = np.array([
    [(my_id,my_id), 1, False], 
    [(19364,my_id), 1, True],
    [(8302,my_id), 1, True],   
    [(18352,my_id), 1, True],   
    [(10319,my_id), 1, True],   
    [(6599,my_id), 1, True],   
    [(20079,my_id), 1, True],   
              ])

indices = my_node[:,0].astype(bwp.INDICES_DTYPE)
data = my_node[:,1].astype(float)
flip = my_node[:,2].astype(bool)

/tmp/ipykernel_32564/2455489822.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  my_node = np.array([


In [13]:
## Creating the node representing "Operational Energy Moerschberg for 1 year"
foreground = bwp.create_datapackage(
fs = bwp.generic_zipfile_filesystem(dirpath=Path("./data/inputs"), filename="foreground.zip", write=True)
)

foreground.add_persistent_vector(
    matrix="technosphere_matrix",
    data_array=data,
    indices_array=indices,
    flip_array=flip,
    name="example",
)

In [14]:
resource=foreground.resources[0]
if resource["matrix"] == "technosphere_matrix" and resource["kind"]=="indices":
    indices,_ = foreground.get_resource(resource["name"])
    unique_indices = set([a for b in indices for a in b])

In [15]:
unique_indices

{6599, 8302, 10319, 18352, 19364, 20079, 1000000}

In [16]:
from utils import database_creator
dict_metadata = {
    100000:{
        "name":"Production of example",
        "code":"Production of example".replace(" ","_"),
        "location":"CH",
        "unit":"kg"

    }
}
database_creator(db_name="terrible_example",data_objs=[foreground], dict_metadata=dict_metadata)

In [17]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ei 3.8 cutoff
	terrible_example

In [20]:
list(bd.Database("terrible_example"))

[]

In [19]:
lca = bc.LCA({my_id: 1}, data_objs=[data_objs_ei38] + method_test + [dp])
lca.lci()
lca.lcia()
lca.score

NameError: name 'dp' is not defined

In [37]:
# lca = bc.LCA({19364: 1}, data_objs=[data_objs_ei38] + method_test)
# lca.lci()
# lca.lcia()
# lca.score

0.004655632602942411

In [55]:
foreground.finalize_serialization()